# Data Aggregation and GroupOperations

After Loading, Merging, Preparing data set, we often need to compute group statistics or pivot tables for reporting or visualization.

**Overview**:
* Split pandas object into pieces 
* Computing group sumary statitics: count, mean, std, or user-defined functions.
* Applying function to each column of DataFrame
* Apply within-group transformations or other manipulations, like normalization,linear regression, rank, or subset selection
* Computing pivot tables
* Perform quantile analysis and other data-derived group analyses

# GroupBy Mechanics
As [Hadley Wickham](https://en.wikipedia.org/wiki/Hadley_Wickham) said, group is the same workflow: **split-apply-combine**
![split-apply-combine](https://image.slidesharecdn.com/slides-151008060416-lva1-app6892/95/pandas-powerful-data-analysis-tools-for-python-19-638.jpg?cb=1444284343)

Each grouping key can take many forms, and the keys do not have to be all of the sametype:
* A list or array of values that is the same length as the axis being grouped
* A value indicating a column name in a DataFrame
* A dict or Series giving a correspondence between the values on the axis being grouped and the group names
* A function to be invoked on the axis index or the individual labels in the index

In [1]:
import pandas as pd
from pandas import DataFrame
from pandas import Series
import numpy as np

In [7]:
df = DataFrame({
        'key1': ['a', 'a', 'b', 'b', 'a'],
        'key2': ['one', 'two', 'one', 'two', 'one'],
        'data1': np.random.rand(5),
        'data2': np.random.rand(5)
    })
df

,data1,data2,key1,key2
0,0.349631,0.840004,a,one
1,0.747123,0.908781,a,two
2,0.816156,0.248073,b,one
3,0.588410,0.283334,b,two
4,0.309779,0.124273,a,one


Suppose we want to compute the **mean** of data1 column by using groups labels from **key1**

In [8]:
grouped = df['data1'].groupby(df['key1'])

The importantthing here is that the data (a Series) has been aggregated according to the group key, producing a new Series that is now indexed by the unique values in the key1 column.

In [9]:
grouped.mean()

key1
a    0.468844
b    0.702283
Name: data1, dtype: float64

If instead we had passed multiple arrays as a list

In [14]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.329705
      two     0.747123
b     one     0.816156
      two     0.588410
Name: data1, dtype: float64

we grouped the data using two keys, and the resulting Series now has a hierarchical index consisting of the unique pairs of keys observed. So we can **unstack hirrachical Series to get a DataFrame**

In [15]:
means.unstack()

key2,one,two
key1,,
a,0.329705,0.747123
b,0.816156,0.588410


We can not only group Series by Series key, but also group by an array

In [16]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2005, 2005, 2006])


In [19]:
df['data1'].groupby([states, years]).mean()

California  2005    0.781640
Ohio        2005    0.469020
            2006    0.309779
Name: data1, dtype: float64

Frequently the grouping information to be found in the same DataFrame as the data you want to work on

In [20]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.468844,0.624352
b,0.702283,0.265704


In [22]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.297860  0.232113
     two   0.065576  0.529084
b    one   0.084591  0.687881
     two   0.483881  0.437802

## Iterating Over Groups

> The GroupBy object supports iteration, generating a sequence of 2-tuples containing
the group name along with the chunk of data


In [21]:
df

,data1,data2,key1,key2
0,0.349631,0.840004,a,one
1,0.747123,0.908781,a,two
2,0.816156,0.248073,b,one
3,0.588410,0.283334,b,two
4,0.309779,0.124273,a,one


In [26]:
df.groupby('key1')

Groupby divide dataset in to chunks of data, and each of them can be access by **iterator**

In [28]:
for group1, group2  in df.groupby('key1'):
    print group1
    print group2

a
      data1     data2 key1 key2
0  0.349631  0.840004    a  one
1  0.747123  0.908781    a  two
4  0.309779  0.124273    a  one
b
      data1     data2 key1 key2
2  0.816156  0.248073    b  one
3  0.588410  0.283334    b  two


In [30]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.329705  0.482138
     two   0.747123  0.908781
b    one   0.816156  0.248073
     two   0.588410  0.283334

In the case of multiple keys

In [42]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print k1, k2
    print "==========="
    print group 
    print "==========="

a one
      data1     data2 key1 key2
0  0.349631  0.840004    a  one
4  0.309779  0.124273    a  one
a two
      data1     data2 key1 key2
1  0.747123  0.908781    a  two
b one
      data1     data2 key1 key2
2  0.816156  0.248073    b  one
b two
     data1     data2 key1 key2
3  0.58841  0.283334    b  two


Of course, we can choose to do whatever you want with the pieces of data. Such as computing a dict

In [43]:
pieces = dict(list(df.groupby('key1')))
pieces

{'a':       data1     data2 key1 key2
 0  0.349631  0.840004    a  one
 1  0.747123  0.908781    a  two
 4  0.309779  0.124273    a  one, 'b':       data1     data2 key1 key2
 2  0.816156  0.248073    b  one
 3  0.588410  0.283334    b  two}

By default groupby groups on **axis=0**. We can change to groupby other axis

In [51]:
df

,data1,data2,key1,key2
0,0.349631,0.840004,a,one
1,0.747123,0.908781,a,two
2,0.816156,0.248073,b,one
3,0.588410,0.283334,b,two
4,0.309779,0.124273,a,one


In [50]:
df.dtypes

data1    float64
data2    float64
key1      object
key2      object
dtype: object

In [52]:
grouped = df.groupby(df.dtypes, axis=1)
dict(list(grouped))

{dtype('float64'):       data1     data2
 0  0.349631  0.840004
 1  0.747123  0.908781
 2  0.816156  0.248073
 3  0.588410  0.283334
 4  0.309779  0.124273, dtype('O'):   key1 key2
 0    a  one
 1    a  two
 2    b  one
 3    b  two
 4    a  one}

## Selecting a Column or Subset of Columns